# Part 2: Ray Data Architecture and Optimization

**Time to complete**: 25 min | **Difficulty**: Advanced | **Prerequisites**: Complete Part 1 and Part 2

**[← Back to Part 1](01-inference-fundamentals.ipynb)** | **[Return to Overview](README.md)**

---

## What You'll Learn

Understanding Ray Data's architecture is critical for making informed optimization decisions. In this part, you'll learn:

1. How Ray Data's streaming execution model enables efficient batch inference
2. How blocks and the object store affect memory management
3. How operators and planning impact performance
4. How resource management affects optimization strategies
5. Why these architectural choices matter for your inference workloads

## Prerequisites

Complete [Part 1: Inference Fundamentals](01-inference-fundamentals.md) and [Part 2: Advanced Optimization](02-advanced-optimization.md) before starting this part.

## Table of Contents

1. [Streaming Execution Model](#streaming-execution-model)
2. [Datasets and Blocks](#datasets-and-blocks)
3. [Ray Memory Model](#ray-memory-model)
4. [Operators and Planning](#operators-and-planning)
5. [Resource Management](#resource-management-and-backpressure)
6. [Optimization Implications](#optimization-implications)

---

## Streaming Execution Model

## Streaming execution

Ray Data processes large datasets efficiently using a streaming model, which works with **blocks** as the basic units of data.

This approach replaces traditional bulk processing, where the entire dataset and intermediate results had to fit in the cluster's memory.

Ray Data's streaming execution combines the **best of batch processing techniques** (logical and physical plan optimizations) along with the **best of data stream processin**g (high scalability and concurrent stages)


### Why Streaming Execution Matters for Batch Inference

Traditional batch processing loads entire datasets into memory before processing. For batch inference with 1M+ images or documents, this approach fails:

**Traditional Batch Processing Problems:**
- **Memory explosion**: Loading 1M images × 3MB each = 3TB memory required
- **No pipeline parallelism**: Model loading, preprocessing, inference all sequential, GPUs just waiting idle
- **Long time to first result**: Wait for all data to load before any inference
- **OOM errors**: Cluster runs out of memory frequently

**Ray Data Streaming Execution Solution:**
- **Constant memory**: Process 128MB blocks at a time, not full dataset
- **Pipeline parallelism**: Load, preprocess, and infer simultaneously
- **Fast time to first result**: Start inferring as soon as first block loads, fully saturating your GPUs
- **Automatic backpressure**: Prevents memory overflow dynamically


### Visualizing the Difference

**Traditional Batch Processing:**

<img src="https://anyscale-materials.s3.us-west-2.amazonaws.com/cko-2025-q1/batch-processing.png" width="800" alt="Traditional Batch Processing">

**Problems with traditional approach:**
- High memory - requires loading entire dataset
- No parallelism - stages run sequentially  
- Long latency - wait for complete load before processing
- Wasted resources - GPUs idle during load/write stages

**Ray Data Streaming Execution:**

<img src="https://anyscale-materials.s3.us-west-2.amazonaws.com/cko-2025-q1/pipelining.png" width="800" alt="Ray Data Streaming Execution">

**Benefits of streaming execution:**
- Low memory - constant predidctable blocks regardless of dataset size
- Pipeline parallelism - all stages active simultaneously
- Fast first result - inference starts immediately
- Maximum throughput - all resources utilized continuously

### How This Affects Your Optimization Decisions

Understanding streaming execution helps you make better optimization choices:

**1. Batch Size Selection:**
- **Don't make batch_size too large**: Risk memory overflow
- **Don't make batch_size too small**: Waste GPU capacity
- **Sweet spot**: Match GPU memory and throughput needs

**2. Concurrency Configuration:**
- **Too many actors**: Backpressure kicks in, actors idle waiting for resources
- **Too few actors**: Underutilized cluster, low throughput
- **Optimal**: Match available GPUs and memory constraints

**3. Model Loading Strategy:**
- **Why actors work**: Model loads once, reused across many blocks
- **Why tasks fail**: Model reloads for every block, massive overhead
- **Architecture enables**: Stateful processing without memory bloat

### Practical Example

In [1]:
import ray
from typing import Dict, Any
import numpy as np
import torch
from torchvision.models import ResNet152_Weights
from torchvision import transforms
from torchvision import models
import ray.data

# Check if GPU is available
HAS_GPU = torch.cuda.is_available()

def preprocess_image(row: Dict[str, np.ndarray]) -> Dict[str, np.ndarray]:
    weights = ResNet152_Weights.IMAGENET1K_V1
    imagenet_transforms = weights.transforms()
    transform = transforms.Compose([
        transforms.ToTensor(),
        imagenet_transforms
    ])
    transformed_tensor = transform(row["image"])

    return {
        "transformed_image": transformed_tensor.numpy(),
    }


# InferenceWorker from Notebook 1
class InferenceWorker:

    def __init__(self):
        self.weights = ResNet152_Weights.IMAGENET1K_V1
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = models.resnet152(weights=self.weights).to(self.device)
        self.model.eval()
    
        imagenet_transforms = self.weights.transforms()
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            imagenet_transforms
        ])

    def __call__(self, batch: Dict[str, np.ndarray]) -> Dict[str, Any]:
        torch_batch = torch.from_numpy(batch["transformed_image"]).to(self.device)
        
        with torch.no_grad():
            prediction = self.model(torch_batch)
            predicted_classes = prediction.argmax(dim=1).detach().cpu()
            predicted_labels = [
                self.weights.meta["categories"][i] for i in predicted_classes
            ]
        
            probabilities = torch.nn.functional.softmax(prediction, dim=1)
            confidence_scores = probabilities.max(dim=1).values.detach().cpu().numpy()
        
        return {
            "predicted_label": predicted_labels,
            "confidence": confidence_scores.tolist(),
        }

# Example: How streaming execution enables large-scale inference
images = ray.data.read_images(
    "s3://anonymous@air-example-data-2/imagenette2/train/",
    mode="RGB",
    ray_remote_args={"num_cpus": 0.1},
).limit(1000)

# This works even if cluster only has 64GB memory!
# Why? Streaming execution processes 128MB blocks at a time
results = images.map(preprocess_image).map_batches(
    InferenceWorker,  # Loads once per actor
    batch_size=32,   # Small batches prevent memory overflow
    # num_gpus=1,      # One model per GPU
    concurrency=4    # 4 parallel actors
)

# As you iterate results, Ray Data:
# 1. Loads blocks from S3 (streaming)
# 2. Preprocesses in parallel (pipeline parallelism)
# 3. Runs inference on GPUs (distributed)
# 4. Writes results (continuous output)
# All while maintaining constant memory footprint!

for batch in results.iter_batches(batch_size=10):
    # First results available immediately
    # Don't need to wait for all 1M images
    print(batch)

2025-10-23 17:03:04,172	INFO worker.py:1833 -- Connecting to existing Ray cluster at address: 10.0.35.66:6379...
2025-10-23 17:03:04,184	INFO worker.py:2004 -- Connected to Ray cluster. View the dashboard at https://session-d1zksaq9ieevgqna5eqcl23rjk.i.anyscaleuserdata-staging.com 
2025-10-23 17:03:04,187	INFO packaging.py:380 -- Pushing file package 'gcs://_ray_pkg_3bb37a88b8e9fec528627c1380803c9e37219f36.zip' (0.29MiB) to Ray cluster...
2025-10-23 17:03:04,188	INFO packaging.py:393 -- Successfully pushed file package 'gcs://_ray_pkg_3bb37a88b8e9fec528627c1380803c9e37219f36.zip'.
/home/ray/anaconda3/lib/python3.12/site-packages/ray/_private/worker.py:2052: FutureWarning: Tip: In future versions of Ray, Ray will no longer override accelerator visible devices env var if num_gpus=0 or num_gpus=None (default). To enable this behavior and turn off this error message, set RAY_ACCEL_ENV_VAR_OVERRIDE_ON_ZERO=0
  warnings.warn(
2025-10-23 17:03:09,059	INFO logging.py:293 -- Registered dataset 

Running 0: 0.00 row [00:00, ? row/s]

{"asctime":"2025-10-23 17:03:09,190","levelname":"E","message":"Actor with class name: 'MapWorker(Map(preprocess_image)->MapBatches(InferenceWorker))' and ID: '27fd6720fbea830fe8023fee02000000' has constructor arguments in the object store and max_restarts > 0. If the arguments in the object store go out of scope or are lost, the actor restart will fail. See https://github.com/ray-project/ray/issues/53727 for more details.","filename":"core_worker.cc","lineno":2161}


- ListFiles 1: 0.00 row [00:00, ? row/s]

- limit=1000 2: 0.00 row [00:00, ? row/s]

...

(MapWorker(Map(preprocess_image)->MapBatches(InferenceWorker)) pid=3426, ip=10.0.3.18) Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /home/ray/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth [repeated 3x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)


  0%|          | 0.00/230M [00:00<?, ?B/s] [repeated 3x across cluster]=10.0.3.18) 
 89%|████████▉ | 206M/230M [00:00<00:00, 246MB/s] [repeated 24x across cluster].127) 
100%|██████████| 230M/230M [00:01<00:00, 204MB/s](InferenceWorker)) pid=3426, ip=10.0.3.18) 


{'predicted_label': array(['tench', 'tench', 'tench', 'English springer', 'English springer',
       'English springer', 'English springer', 'English springer',
       'radio', 'tape player'], dtype=object), 'confidence': array([0.99935967, 0.99985158, 0.99108666, 0.49943095, 0.99055171,
       0.99199039, 0.85377097, 0.69636279, 0.36682165, 0.54299575])}
{'predicted_label': array(['loudspeaker', 'CD player', 'sports car', 'chain saw', 'chain saw',
       'chain saw', 'chain saw', 'chain saw', 'church', 'altar'],
      dtype=object), 'confidence': array([0.84784979, 0.93608212, 0.44139817, 0.99995494, 0.98789829,
       0.99997699, 0.98841596, 0.99988866, 0.23647654, 0.49457902])}
{'predicted_label': array(['church', 'church', 'French horn', 'French horn', 'French horn',
       'French horn', 'French horn', 'garbage truck', 'garbage truck',
       'garbage truck'], dtype=object), 'confidence': array([0.99217391, 0.49143541, 0.80212873, 0.99511343, 0.99905521,
       0.96243846, 0.99852

2025-10-23 17:06:39,142	INFO streaming_executor.py:279 -- ✔️  Dataset dataset_3_0 execution finished in 210.01 seconds


{'predicted_label': array(['parachute', 'parachute', 'parachute', 'tench', 'tench', 'tench',
       'tench', 'tench', 'tench', 'English springer'], dtype=object), 'confidence': array([0.99843508, 0.99999845, 0.9997533 , 0.92295462, 0.99542964,
       0.99858332, 0.9927972 , 0.9998436 , 0.97728777, 0.80834675])}
{'predicted_label': array(['English springer', 'English springer', 'English springer',
       'English springer', 'cassette', 'cassette player',
       'cassette player', 'cassette player', 'switch', 'chain saw'],
      dtype=object), 'confidence': array([0.72270048, 0.89827406, 0.94940645, 0.6293081 , 0.5389266 ,
       0.55195016, 0.92570847, 0.85766459, 0.34444249, 0.9998554 ])}
{'predicted_label': array(['chain saw', 'chain saw', 'chain saw', 'bell cote', 'church',
       'church', 'church', 'church', 'French horn', 'French horn'],
      dtype=object), 'confidence': array([0.99728942, 0.95580912, 0.99999905, 0.42472515, 0.96667236,
       0.81282824, 0.78124791, 0.98568439, 

---
# Ray Data Architecture Deep Dive

## Datasets and Blocks

### What Are Blocks?

A **block** is the fundamental unit of data in Ray Data. Understanding blocks is essential for optimization.

**Block characteristics:**
- **Size**: Typically 128MB (configurable via `target_max_block_size`). This is the industry standard for partition sizing for the average ratio of CPU to memory to I/O. This value can be adjust via the Ray Data context.
- **Format**: Stored as PyArrow tables or pandas DataFrames internally. This allows for easy integrations with the rest of the Python ecosystem and lower serialization costs compared to JVM-based engines.
- **Location**: Ray object store (shared memory), this builds off of the existing work in Ray Core for fast in-memory trasnfers and data spilling.
- **Processing unit**: One block = one task typically when batch_size=None. Ray Data automatically tries to find the best batch sizing, but can also be configured by the user.

### Why Block Size Matters for Batch Inference

Block size directly impacts inference performance:

**Block Size Too Small (e.g., 1MB):**
- Too many tasks created (scheduling overhead)
- Poor GPU utilization (small batches)
- High network overhead (many small transfers)
- Scheduler bottleneck (managing thousands of tasks)

**Block Size Too Large (e.g., 10GB):**
- Memory pressure (blocks don't fit in object store)
- Poor parallelism (few blocks = few parallel tasks)
- Spilling to disk (performance degradation)
- Uneven load balancing (some workers idle)

**Optimal Block Size (128MB default):**
- Good parallelism (many blocks for distribution)
- Low overhead (reasonable number of tasks)
- Fits in memory (object store can hold multiple blocks)
- Efficient transfer (network overhead manageable)

### Configuring Block Size for Inference

In [2]:
import ray

# Configure block size for your inference workload
ctx = ray.data.DataContext.get_current()

# Default: 128MB blocks
print(f"Default max block size: {ctx.target_max_block_size / 1024**2:.0f}MB")

# For image inference with large images:
# Smaller blocks = more parallelism
ctx.target_max_block_size = 64 * 1024**2  # 64MB blocks

# For text inference with small documents:
# Larger blocks = less overhead
ctx.target_max_block_size = 256 * 1024**2  # 256MB blocks

# Load images with configured block size
images = ray.data.read_images(
    "s3://anonymous@air-example-data-2/imagenette2/train/",
    mode="RGB",
    ray_remote_args={"num_cpus": 1},
).limit(1000).materialize()

print(f"Dataset blocks: {images.num_blocks()}")
print(f"Estimated blocks: {images.size_bytes() / ctx.target_max_block_size:.0f}")
print(f"Dataset stats: {images.stats()}")

2025-10-23 17:06:39,281	INFO logging.py:293 -- Registered dataset logger for dataset dataset_6_0
2025-10-23 17:06:39,286	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_6_0. Full logs are in /tmp/ray/session_2025-10-23_16-34-47_857542_2333/logs/ray-data
2025-10-23 17:06:39,286	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_6_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> LimitOperator[limit=1000] -> TaskPoolMapOperator[ReadFiles]


Default max block size: 128MB


Running 0: 0.00 row [00:00, ? row/s]

- ListFiles 1: 0.00 row [00:00, ? row/s]

- limit=1000 2: 0.00 row [00:00, ? row/s]

- ReadFiles 3: 0.00 row [00:00, ? row/s]

2025-10-23 17:06:43,642	INFO streaming_executor.py:279 -- ✔️  Dataset dataset_6_0 execution finished in 4.35 seconds


Dataset blocks: 22
Estimated blocks: 3
Dataset stats: Operator 0 ReadFiles: 

Operator 1 ListFiles: 1 tasks executed, 200 blocks produced in 1.47s
* Remote wall time: 96.02us min, 1.17s max, 5.96ms mean, 1.19s total
* Remote cpu time: 95.41us min, 440.35ms max, 2.33ms mean, 465.25ms total
* UDF time: 0us min, 0us max, 0.0us mean, 0us total
* Peak heap memory usage (MiB): 179.93 min, 180.18 max, 180 mean
* Output num rows per block: 43 min, 54 max, 47 mean, 9469 total
* Output size bytes per block: 3929 min, 4923 max, 4303 mean, 860757 total
* Output rows per task: 9469 min, 9469 max, 9469 mean, 1 tasks used
* Tasks per node: 1 min, 1 max, 1 mean; 1 nodes used
* Operator throughput:
	* Total input num rows: 0 rows
	* Total output num rows: 9469 rows
	* Ray Data throughput: 6431.857209243873 rows/s
	* Estimated single node throughput: 7944.014838581921 rows/s

Operator 2 limit=1000: 1 tasks executed, 22 blocks produced in 1.2s
* Remote wall time: 96.02us min, 1.17s max, 53.34ms mean, 1.1

### How Blocks Flow Through Inference Pipeline

Ray Data's block-based architecture enables parallelism at every stage:

<img src="https://docs.ray.io/en/latest/_images/dataset-arch.svg" width="700" alt="Ray Data Block Architecture">

**Key insights:**
- Each block contains a disjoint subset of rows
- Blocks are processed in parallel across the cluster
- Distributed object store enables efficient block transfer
- Tasks operate on individual blocks for maximum parallelism

---

## Ray Memory Model

### Object Store and Heap Memory

Ray manages two types of memory that affect batch inference:

<img src="https://docs.ray.io/en/latest/_images/memory.svg" width="600" alt="Ray Memory Model">

**1. Object Store Memory (30% of node memory by default, set to a higher ratio for Ray Data):**
- **Purpose**: Shared memory for passing data between tasks
- **Contents**: Blocks (PyArrow tables), task outputs, intermediate results
- **Optimization impact**: Determines how many blocks can be in-flight
- **When full**: Triggers spilling to disk (major performance hit)

**2. Heap Memory (70% of node memory by default):**
- **Purpose**: Task execution, model loading, preprocessing
- **Contents**: Loaded models, batch data being processed, Python objects
- **Optimization impact**: Determines how many models fit in memory
- **When full**: Python out-of-memory errors, task failures

<div style="border: 2px solid #ffeb3b; background: #fffde7; border-radius: 6px; padding: 18px 18px 12px 18px; margin-bottom: 18px;">
  <strong>💡 <span style="color:#555;">Ray Data Optimization Tip</span>:</strong>
  <ul style="margin:8px 0 8px 20px;">
    <li>
      <b>Set the <code>object store memory</code> for Ray Data workloads to <span style="color:#002984;">at least <u>50%</u> of your node memory</span></b>.
    </li>
    <li>
      Ray's <code>object_store_memory</code> default (30%) is often <u>tuned for Ray Core, Train, or compute-heavy workloads</u> with large heap requirements.
    </li>
    <li>
      <b>Ray Data</b> workloads depend on storing <b>blocks of data in the object store</b> for maximum parallel throughput and efficiency.
    </li>
    <li>
      <b>Increase the object store allocation</b> for Ray Data so that your pipeline can hold more data in-memory without frequent spilling to disk or throughput collapse.
    </li>
  </ul>
  <span style="color:#444;">See <code>object_store_memory</code> and <code>DataContext</code> configuration for details.</span>
</div>


### How This Affects Batch Inference Optimization

```text
Node Memory: 64GB
├── Object Store (30% = 19GB)
│   ├── Block 1 (128MB)
│   ├── Block 2 (128MB)
│   ├── ...
│   └── Block N (up to ~148 blocks fit)
│
└── Heap Memory (70% = 45GB)
    ├── Model weights (5GB per model)
    ├── Batch preprocessing (2GB per actor)
    ├── Python overhead (1GB)
    └── Available for actors: ~37GB
        → Can fit ~7 model actors at 5GB each
```

**Optimization implications:**

**Object Store Pressure:**
- **Symptom**: "Object store full" warnings in logs
- **Cause**: Too many blocks generated too fast
- **Solution**: Reduce `concurrency` or modify `num_cpus` on the reader tasks

**Heap Memory Pressure:**
- **Symptom**: Out-of-memory or GRAM errors, task failures
- **Cause**: Too many models loaded or batch_size too large
- **Solution**: Reduce `concurrency` or `batch_size`

### Practical Memory Configuration

```python
import ray

# Set up Ray with a custom object store size.
# This reserves 4 GB (4 * 1024**3 bytes) exclusively for Ray's plasma object store.
ray.init(object_store_memory=4 * 1024**3)

# Set Ray Data to be allowed to use up to 75% of the object store for its block cache.
# There are two ways to do this: via env var or programmatically.
import ray.data

# Option 1: Environment variable (do before process start, not here)
# os.environ["RAY_DATA_OBJECT_STORE_MEMORY_LIMIT_FRACTION"] = "0.75"

# Option 2: Programmatically set the fraction (recommended in Ray 2.x+)
ctx = ray.data.DataContext.get_current()
ctx.object_store_memory_limit_fraction = 0.75

# What does this mean?
# - Total node memory:          (e.g., 64GB)
# - Ray object store:           4GB  (from ray.init above)
# - Ray Data can use up to 75%: 3GB  (0.75 * 4GB) for its block cache.
# - Other Ray workloads (core, RLlib, actors, etc.) share the remaining object store (up to 1GB).

# This ensures Ray Data can't overwhelm the object store and will apply backpressure if it fills its quota,
# helping to prevent "object store full" errors and OOMs.

# Example Ray Data pipeline using the above memory configuration:
images = ray.data.read_images("s3://images/", num_cpus=0.05)

results = images.map_batches(
    LargeModelInference,   # Example: Large model, may require lots of heap memory
    batch_size=16,         # Small batches if GPU is a bottleneck
    num_gpus=1,
    concurrency=2          # Only 2 concurrent actors to avoid heap pressure
)

# Ray Data will:
# - Backpressure file reads and ingestion when its allocated share (3GB here) fills up
# - Limit concurrent tasks to avoid heap and object store OOM
# - Enable more predictable, tunable memory usage for batch inference
```

### Zero-Copy Optimization

Ray Data uses **zero-copy deserialization** for efficiency:

**What it means:**
- Blocks stored in object store are PyArrow tables
- Accessing a block doesn't copy data - just creates a pointer
- Multiple tasks can read same block without duplication

**Why it matters for inference:**
- **Memory efficiency**: 10 actors can share same preprocessed block
- **Performance**: No serialization overhead between stages
- **Scalability**: Enables high-throughput pipelines

**Practical impact:**

In [3]:
# Without zero-copy (hypothetical):
# Block in object store: 128MB
# Actor 1 reads block: +128MB copy → 256MB total
# Actor 2 reads block: +128MB copy → 384MB total
# Result: 3x memory usage!

# With zero-copy (Ray Data actual):
# Block in object store: 128MB
# Actor 1 reads block: 0MB copy (pointer) → 128MB total
# Actor 2 reads block: 0MB copy (pointer) → 128MB total
# Result: Constant memory!

**Visual representation of object store usage:**

<img src="https://anyscale-materials.s3.us-west-2.amazonaws.com/ray-data-deep-dive/producer-consumer-object-store-v2.png" width="700" alt="Object Store Data Flow">

---

## Operators and Planning

### Logical vs Physical Plans

Ray Data transforms your code into an optimized execution plan:

**Your Code:**
```python
results = (
    ray.data.read_images("s3://images/")
    .map_batches(preprocess_images)
    .map_batches(InferenceModel, num_gpus=1)
)
```

### Planning Steps

Below are the steps that Ray Data takes to plan the execution of a dataset.

1. Ray Data optimizes the logical plan performing a series of optimizations to produce an optimized logical plan. 

2. The plan then gets translated into a physical plan using a physical planner

3. The physical plan is then further optimized (e.g. fusing operators) to produce an optimized physical plan.

See the below diagram for the planning process:

<img src="https://anyscale-materials.s3.us-west-2.amazonaws.com/ray-data-deep-dive/get_execution_plan.png" width="600">


**Logical Plan (What to do):**
```
ReadFiles → MapBatches[preprocess] → MapBatches[inference]
```

**Physical Plan (How to do it):**
```
TaskPoolMapOperator[ReadFiles→preprocess→inference]
```

**Note the fusion:** Ray Data combined all three operations into a single operator!

### Operators

The DAG shows that "logically" we require operations like `ReadFiles` and `MapBatches` to be run.

Physical plans indicate "how" the logical operators will be executed.

The DAG shows a plan of physical operators that will be executed:
- `InputDataBuffer` is a placeholder for the input data.
- `TaskPoolMapOperator` is a physical operator which will execute logical operations like `ReadFiles` and `MapBatches` using a "pool" of Ray Tasks. 
- The syntax is `{PhysicalOperator}[{LogicalOperator}]`. 

#### Operator Fusion
Under certain conditions, Ray Data will fuse operators together to reduce data movement and improve execution efficiency.

Here is the syntax for fusing operators:
- `{PhysicalOperator}[{LogicalOperator1}->{LogicalOperator2}]`

### Operator Fusion and Its Impact on Inference

**Operator fusion** combines multiple operations into single tasks to reduce overhead.

**Benefits for batch inference:**
- **Reduced data movement**: Preprocessed images go straight to model (no object store roundtrip)
- **Lower task overhead**: One task instead of three per block
- **Better GPU utilization**: Continuous processing without gaps
- **Memory efficiency**: Intermediate results stay in task memory

**When fusion happens:**
- Same compute configuration (both use tasks or both use actors)
- Compatible batch sizes
- Compatible resource requirements
- No shuffle/repartition operations between them

**When fusion doesn't happen:**
- Different compute strategies (task vs actor)
- Different resource requirements (CPU vs GPU)
- Shuffle operations (groupby, sort, repartition)

**Optimization strategy:**
Keep preprocessing and inference configs compatible to enable fusion:
```python
# Good: Fusion enabled
images.map_batches(preprocess, batch_size=32).map_batches(InferenceModel, batch_size=32

# Suboptimal: Fusion disabled (different batch sizes)
images.map_batches(preprocess, batch_size=64).map_batches(InferenceModel, batch_size=32)
```

## Streaming Topology

After constructing the optimized DAG of physical operators, the execution plan is handed to the `StreamingExecutor` to execute.

The first step is to build a streaming topology, here is a sample diagram of the streaming topology:

<img src="https://anyscale-materials.s3.us-west-2.amazonaws.com/ray-data-deep-dive/build_streaming_topology.png" width="1000">

Each physical operator will be wired to the next physical operator downstream in the execution plan. 

An upstream operator's external output queue will *refer to the same queue* as the input of the downstream operator.

## Data flow within an operator

Below is a diagram showing the data flow within an operator.

<img src="https://anyscale-materials.s3.us-west-2.amazonaws.com/ray-data-deep-dive/data_flow_simplified_v4.png" width="800">


----

# Optimizing Batch Inference

## Resource Management and Backpressure

### Dynamic Resource Allocation

Ray Data automatically manages resources across operators to maximize throughput. Understanding this helps you set optimal parameters.

**The Challenge:**
- **Too aggressive loading**: Object store fills up, spilling to disk
- **Too conservative loading**: GPUs idle waiting for data
- **Unbalanced pipeline**: Some stages bottleneck while others wait

**Ray Data's Solution:**
Dynamically allocates resources based on operator throughput and backpressure policies.

### Backpressure Mechanisms

**1. Submission-Based Backpressure:**
Prevents operators from submitting new tasks when resource budgets exceeded.

**Example scenario:**
```
GPU inference slower than data loading
↓
Object store filling with preprocessed images
↓
Ray Data backpressures data loading
↓
Loading slows down to match inference throughput
↓
Balanced pipeline - no memory overflow
```

**2. Output-Based Backpressure:**
Limits how many task outputs move to operator queues based on memory availability.

**Practical impact on inference:**

```python
# Scenario: Fast preprocessing, slow inference

images = ray.data.read_images("s3://images/", num_cpus=0.05)

results = images.map_batches(
    fast_preprocess,     # Processes 1000 images/sec
    batch_size=64,
    concurrency=16        # Many parallel preprocessors
).map_batches(
    SlowInferenceModel,  # Processes 100 images/sec
    batch_size=16,
    num_gpus=1,
    concurrency=2         # Only 2 GPUs available
)
```

What Ray Data does automatically:
1. Preprocessing generates blocks faster than inference consumes
2. Object store starts filling with preprocessed blocks
3. Backpressure kicks in - preprocessing tasks not scheduled
4. Pipeline balances - preprocessing matches inference rate
5. Memory stays constant - no overflow despite throughput mismatch


## Resource management and allocation

Ray data will:
- dynamically allocate resources across operators
- backpressure operators that have exceeded their resource budgets

### Operator backpressure in Ray Data

Backpressure in Ray Data is essential for managing resource contention and ensuring fair resource utilization across operators. Its primary goal is to maximize execution plan throughput by controlling task flow.

### How Backpressure Works

Backpressure can be applied to each physical operator in an execution plan through two approaches:

1. **Submission-Based Backpressure**: Prevents an operator from submitting new tasks if it exceeds its resource budget.
2. **Output-Based Backpressure**: Prevents an operator from moving outputs to its out-queue if it produces more data than its resource budget allows.

### Where is backpressure applied?

Below is a diagram of the scheduling loop highlighting where backpressure is applied.

<img src="https://anyscale-materials.s3.us-west-2.amazonaws.com/ray-data-deep-dive/scheduling_loop_with_backpressure.png" width="1000">

<details>
<summary>Here is how backpressure is implemented:</summary>

##### Submission-based backpressure
Submission-based backpressure is implemented in the `ResourceManager.can_submit_new_task()` method. It is used in the `select_operator_to_run()` method to determine if an operator can submit new tasks.

##### Output-based backpressure

Output-based backpressure is implemented in the `ResourceManager.max_task_output_bytes_to_read()` method. It is used in the `process_completed_tasks()` method to determine the maximum bytes of task outputs that can be read and moved to an external queue.

<details>
<summary>Here are the technical details where output-based backpressure is applied:</summary>

The `scheduling_loop_step()` will invoke:

- `process_completed_tasks()` to:
  - Wait and gather completed task outputs from all operators
  - Move completed task outputs to the operator's in-queue

`process_completed_tasks` will make use of `max_task_output_bytes_to_read()` to determine the maximum bytes of task outputs that can be read and moved to an external queue.

Here is the code for `process_completed_tasks()`:

```python
from ray.data._internal.execution.streaming_executor_state import process_completed_tasks

%psource process_completed_tasks
```
</details>



### Monitoring Resource Usage
```python
# Enable Ray Data resource manager debug logging
import os
os.environ['RAY_DATA_DEBUG_RESOURCE_MANAGER'] = '1'

# Run your inference pipeline
results = images.map_batches(InferenceModel, num_gpus=1, concurrency=4)
```
You'll see output like:
```text
[ResourceManager] Operator budgets:
  ReadImages: object_store_memory=5.0GB, cpu=8.0
  MapBatches: object_store_memory=5.0GB, cpu=0.0, gpu=4.0
  MapBatches: object_store_memory=10.0GB, cpu=0.0, gpu=0.0
```
This shows:
- How resources are allocated across operators
- Where bottlenecks might occur
- If backpressure is active

---

## Optimization Implications

### How Architecture Informs Optimization Decisions

Understanding Ray Data's architecture helps you make better optimization choices:

#### 1. Choosing Batch Size

**Architectural considerations:**
- **Block size**: Batch size should divide evenly into block size for efficiency
- **GPU memory**: Batch must fit in GPU memory during inference
- **Object store**: Preprocessed batches must fit in object store
- **Throughput**: Larger batches = better GPU utilization (up to a point)

<div class="alert alert-info">
    <b>Note:</b> Individual configurations will vary between applications, these examples configs are for showcasing example logic that may be applied
</div>

**Example Decision framework:**

In [5]:
# Calculate optimal batch size

# Factor 1: GPU memory constraint
gpu_memory_gb = 16  # Your GPU memory
model_size_gb = 5
batch_overhead_mb = 50  # Per sample
max_batch_from_gpu = int((gpu_memory_gb - model_size_gb) * 1024 / batch_overhead_mb)

# Factor 2: Block size alignment
block_size_mb = 128
samples_per_block = block_size_mb / 3  # 3MB per image
ideal_batch_for_blocks = int(samples_per_block / 4)  # 4 batches per block

# Factor 3: Throughput testing
# Test different sizes: 16, 32, 64, 128
# Choose largest that doesn't cause memory issues

# Final choice: Minimum of all constraints
optimal_batch_size = min(max_batch_from_gpu, ideal_batch_for_blocks, 128)
print(f"Optimal batch size: {optimal_batch_size}")

Optimal batch size: 10


#### 2. Choosing Concurrency

**Architectural considerations:**
- **GPU count**: One actor per GPU maximum
- **Memory per actor**: Model size + batch size determines how many actors fit
- **Object store capacity**: More actors = more in-flight blocks
- **CPU availability**: Preprocessing may need CPUs

**Example Decision framework:**

In [6]:
# Calculate optimal concurrency

# Factor 1: GPU constraint
num_gpus = 8  # Available GPUs
max_concurrency_gpu = num_gpus  # One model per GPU

# Factor 2: Memory constraint
node_heap_memory_gb = 64 * 0.7  # 70% of 64GB node
model_size_gb = 5
batch_memory_gb = 2
memory_per_actor = model_size_gb + batch_memory_gb
max_concurrency_memory = int(node_heap_memory_gb / memory_per_actor)

# Factor 3: Object store constraint
object_store_gb = 64 * 0.3  # 30% of 64GB node
block_size_gb = 0.128
blocks_in_flight_per_actor = 2  # Preprocessing + inference
required_object_store = max_concurrency_gpu * blocks_in_flight_per_actor * block_size_gb
max_concurrency_object_store = int(object_store_gb / (blocks_in_flight_per_actor * block_size_gb))

# Final choice: Minimum of all constraints
optimal_concurrency = min(
    max_concurrency_gpu,
    max_concurrency_memory,
    max_concurrency_object_store
)

print(f"Optimal concurrency: {optimal_concurrency}")
print(f"  GPU limit: {max_concurrency_gpu}")
print(f"  Memory limit: {max_concurrency_memory}")
print(f"  Object store limit: {max_concurrency_object_store}")

Optimal concurrency: 6
  GPU limit: 8
  Memory limit: 6
  Object store limit: 75


#### 3. Actor vs Task Decision

**Architectural insight:** Actors are stateful, tasks are stateless.

| Aspect | Tasks | Actors | Best For Inference |
|--------|-------|--------|-------------------|
| **Startup** | Launch per invocation | Launch once, reuse | - Actors (amortize model loading) |
| **State** | Stateless | Stateful | - Actors (keep model in memory) |
| **Resource overhead** | Low | Higher | Depends on model size |
| **Scheduling overhead** | Higher (many tasks) | Lower (few actors) | - Actors (fewer scheduling decisions) |
| **Memory** | Released after task | Held by actor | Tasks if memory-constrained |

**For batch inference:** Almost always use actors because:
- Model loading is expensive (2-5 seconds)
- Models are large (500MB - 10GB)
- Reducing load cost across 1000s of batches is critical

#### 4. Understanding Performance Bottlenecks

**Use Ray Dashboard to identify architectural bottlenecks:**

**Symptom 1: Low GPU utilization**
- **Possible cause**: Object store full (loading backpressured)
- **Solution**: Increase object store limit or reduce block size
- **How to verify**: Check "Ray Data Metrics (Object Store Memory)"

**Symptom 2: Spilling to disk**
- **Possible cause**: Too many concurrent actors generating blocks
- **Solution**: Reduce concurrency or increase batch_size
- **How to verify**: Check "Spilled" metric in object store

**Symptom 3: High task overhead**
- **Possible cause**: Blocks too small, too many tasks
- **Solution**: Increase target_max_block_size
- **How to verify**: Check task count vs throughput

**Symptom 4: Actors idle**
- **Possible cause**: Upstream loading too slow
- **Solution**: Increase num_cpus for read operation
- **How to verify**: Check "Ray Data Metrics (Inputs)" throughput

---

## Key Takeaways: Architecture and Optimization

### Critical Architecture Concepts

**1. Streaming Execution:**
- Enables processing datasets larger than cluster memory
- Provides pipeline parallelism for maximum throughput
- Makes batch inference scalable from 1K to 1B samples

**2. Blocks:**
- 128MB default size balances parallelism and overhead
- More blocks = more parallelism (up to a point)
- Block size affects GPU batch size and task count

**3. Memory Model:**
- Object store (30%) holds blocks and transfers
- Heap memory (70%) runs tasks and loads models
- Both limits constrain concurrency and batch size

**4. Operator Fusion:**
- Combines operations to reduce overhead
- Keeps intermediate data in task memory
- Improves throughput and reduces latency

**5. Backpressure:**
- Automatically balances pipeline stages
- Prevents memory overflow
- Maximizes throughput within resource constraints

### Optimization Decision Framework

Use this sample framework informed by Ray Data architecture (just an example, actual configuration will change for different types of applications):

In [7]:
# Step 1: Start with architectural constraints
gpu_count = 8
gpu_memory_gb = 16
node_memory_gb = 64
model_size_gb = 5

# Step 2: Calculate concurrency from memory
heap_memory = node_memory_gb * 0.7
actors_fit = int(heap_memory / (model_size_gb + 2))  # +2GB for batches
concurrency = min(gpu_count, actors_fit)

# Step 3: Calculate batch size from GPU memory
available_gpu_mem = gpu_memory_gb - model_size_gb
sample_size_mb = 3  # Per image
batch_size = int(available_gpu_mem * 1024 / sample_size_mb / 2)  # /2 for safety

# Step 4: Configure block size for efficiency
ctx = ray.data.DataContext.get_current()

# Make blocks contain ~4 batches worth of data
ctx.target_max_block_size = batch_size * sample_size_mb * 1024**2 * 4

# Step 5: Run with optimal settings
# results = images.map_batches(
#     InferenceModel,
#     batch_size=batch_size,
#     num_gpus=1,
#     concurrency=concurrency
# )

print(f"Optimized configuration:")
print(f"  Concurrency: {concurrency} (limited by {min(gpu_count, actors_fit)})")
print(f"  Batch size: {batch_size}")
print(f"  Block size: {ctx.target_max_block_size / 1024**2:.0f}MB")

Optimized configuration:
  Concurrency: 6 (limited by 6)
  Batch size: 1877
  Block size: 22524MB


### Architecture-Aware Performance Tips

**Tip 1: Align batch_size with block_size**

Good configuration:
- Block size: 128MB, batch_size: 32 (4 batches per block - clean division)

Suboptimal configuration:
- Block size: 128MB, batch_size: 50 (2.56 batches per block - awkward division)

**Tip 2: Monitor object store, not just GPUs**

- GPU utilization high but throughput low?
- Check object store - might be spilling to disk
- Ray Dashboard → Metrics → Object Store Memory

**Tip 3: Use fusion-friendly patterns**

```python
# Fusion-friendly: Same compute, compatible configs
images.map_batches(prep, batch_size=32, num_cpus=1).map_batches(model, batch_size=32, num_gpus=1)

# Fusion-incompatible: Different batch sizes  
images.map_batches(prep, batch_size=64).map_batches(model, batch_size=32)
```

**Tip 4: Respect memory limits**
```python
# Set limits based on architecture
ctx.execution_options.resource_limits.object_store_memory = node_memory * 0.3 * 0.5

# This leaves 50% object store for other workloads
# Prevents OOM when running multiple jobs or have memory heavy UDFs
```

---

## Practical Architecture Examples

### Example 1: Small Model, High Throughput

**Scenario:** ResNet-50 (100MB model), process 1M images

**Architectural analysis:**
- **Model size**: Small (100MB) → Many actors fit in memory
- **GPU memory**: 16GB → Large batches possible (128 images)
- **Throughput goal**: Maximize images/second

**Optimal configuration:**

```python
results = images.map_batches(
    ResNet50Model,
    batch_size=128,    # Large batches for throughput
    num_gpus=1,
    concurrency=8      # Use all 8 GPUs
)
```

Why this works:
- Heap memory: 64GB * 0.7 = 45GB
- Per actor: 0.1GB model + 1GB batch = 1.1GB
- Can fit: 45GB / 1.1GB = 40 actors
- Limited by: 8 GPUs → concurrency=8
- Object store: Minimal pressure (small batches)

### Example 2: Large Model, Memory-Constrained

**Scenario:** LLaMA-70B (140GB model with quantization = 35GB), process 100K documents

**Architectural analysis:**
- **Model size**: Huge (35GB) → Very few actors fit
- **GPU memory**: 80GB A100 → Moderate batches (16 documents)
- **Memory goal**: Don't OOM

**Optimal configuration:**
```python
results = documents.map_batches(
    LLaMA70BModel,
    batch_size=16,     # Conservative for large model
    num_gpus=1,
    concurrency=2      # Only 2 models fit in cluster memory
)
```

Why this works:
- Heap memory: 256GB * 0.7 = 179GB (multi-node)
- Per actor: 35GB model + 5GB batch = 40GB
- Can fit: 179GB / 40GB = 4 actors theoretical
- Use: 2 for safety margin (avoid OOM)
- Object store: Backpressure prevents overflow

### Example 3: Balanced Pipeline

**Scenario:** BERT (500MB), preprocessing heavy (embedding generation)

**Architectural analysis:**
- **Preprocessing**: CPU-intensive (tokenization, embedding lookup)
- **Inference**: GPU-intensive (transformer forward pass)
- **Goal**: Balance both stages

**Example Optimal configuration:**

```python
results = (
    documents
    .map_batches(
        heavy_preprocessing,
        batch_size=64,      # CPU batches can be larger
        num_cpus=2,         # Allocate CPUs for preprocessing
        concurrency=16       # Many CPU workers
    )
    .map_batches(
        BERTInference,
        batch_size=32,      # GPU batch size
        num_gpus=1,
        concurrency=4        # 4 GPUs
    )
)
```

Why this works:
- Preprocessing: 16 workers × 2 CPUs = 32 CPUs used
- Inference: 4 workers × 1 GPU = 4 GPUs used
- Backpressure: Automatically balances if mismatch
- Fusion: Disabled (different compute), but that's okay - different resources

---

## Summary: Architecture Drives Optimization

**Key Architectural Principles for Batch Inference:**

1. **Streaming execution** enables unlimited dataset sizes with constant memory
2. **Blocks** are the unit of parallelism - more blocks = more parallel tasks
3. **Object store** holds blocks and transfers - capacity limits in-flight data
4. **Heap memory** holds models and executions - limits concurrent actors
5. **Operator fusion** reduces overhead - keep configs compatible
6. **Backpressure** prevents overflow - trust Ray Data's automatic balancing

**Optimization Strategy:**
1. Start with memory constraints (heap and object store)
2. Calculate maximum concurrency from memory limits
3. Choose batch size for GPU utilization
4. Configure block size for parallelism
5. Monitor Ray Dashboard for bottlenecks
6. Adjust based on observed behavior

---

**[← Back to Part 1](01-inference-fundamentals.ipynb)** | **[Continue to Part 3](03-advanced-optimization.ipynb)** | **[Return to Overview](README.md)**

---

*This architectural deep-dive completes the batch inference optimization series. You now understand not just how to optimize, but why specific optimizations work based on Ray Data's design.*